 Langgraph to differentiate spam from legitimate mails and then drafting response mails for the legitimate ones.

In [2]:
!pip install langgraph


   ------------- -------------------------- 2/6 [langgraph-sdk]
   -------------------- ------------------- 3/6 [langgraph-checkpoint]
   -------------------------- ------------- 4/6 [langgraph-prebuilt]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   --------------------------------- ------ 5/6 [langgraph]
   ---------------------------------------- 6/6 [langgraph]



In [3]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START,END
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

In [4]:
model = ChatOllama(model="llama3", temperature=0)


C:\Users\niran\AppData\Local\Temp\ipykernel_39800\4259199618.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3", temperature=0)


In [5]:
# step 1 define states

class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    spam_reason: Optional[str]
    email_category: Optional[str]
    email__draft: Optional[str]
    messages: List[Dict[str, Any]]

In [6]:
#step 2 Define Nodes

def read_email(state:EmailState):
    email = state["email"]
    print(f"Alfred is proccessing email from \
        {email['sender']} with subject: \
            {email['subject']}")
    return {}

def classify_email(state: EmailState):
    email = state["email"]
    
    prompt= f"""
As Alfred the butler of Mr Wayne (Batman), analyse\
    the email and determine if it is spam or \
        legitimate.

Email:
From: {email['sender']}
Subject: {email['subject']}
Body: {email['body']}

Answer ONLY with 'SPAM' or 'HAM'(legitimate).
"""
    messages= [HumanMessage(content=prompt)]
    response= model.invoke(messages)
    
    response_text = response.content.lower()
    print("Classification result:", response_text)
    is_spam = "spam" in response_text and "ham" not in response_text

    if not is_spam:
        new_messages = state.get("messages", []) + [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content}
        ]
    else:
        new_messages = state.get("messages", [])
    
    return {"is_spam": is_spam, "messages": new_messages}

def handle_spam(state: EmailState):
    print(f"Alfred has marked the email as spam.")
    print(f"The email has been movd to the spam folder.")
    return {}

def drafting_response(state: EmailState):
    email = state['email']
    
    prompt = f"""
As Alfred the butler, draft a polite preliminary \
    response to this email.
    
Email:
From: {email['sender']}
Subject: {email['subjct']}
Body: {email['body']}

Draft a a brief, professional response that Mr wayne\
    can review and personalize.
"""

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    new_messages = state.get("messages", [])+ [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]
    return {"email_draft": response.content, "messages": new_messages}

def notify_mr_wayne(state: EmailState):
    email = state["email"]
    print("\n" + "=" * 50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"\n I've prepared a draft response for your review:")
    print("-"*50)
    print(state["email_draft"])
    print("="*50 + "\n")
    return {}


            


In [7]:
# step 3: Deine Routing Logic

def route_email(state: EmailState)-> str:
    return "spam" if state["is_spam"] else "legitimate"


In [8]:
#step 4: Build LangGraph

email_graph = StateGraph(EmailState)

#add nodes
email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("drafting_response", drafting_response)
email_graph.add_node("notify_mr_wayne", notify_mr_wayne)

#add edges
email_graph.add_edge(START, "read_email")
email_graph.add_edge("read_email", "classify_email")

#add conditional_edges

email_graph.add_conditional_edges(
    "classify_email",
    route_email,
    {
        "spam": "handle_spam",
        "legitimate": "drafting_response"
    }
)

# final edges
email_graph.add_edge("handle_spam", END)
email_graph.add_edge("drafting_response", "notify_mr_wayne")
email_graph.add_edge("notify_mr_wayne", END)

#compile graph
compiled_graph = email_graph.compile()



In [10]:
#step 5 Test with sample Emails

legitimate_email = {
    "sender": "Joker",
    "subject": "Found you Batman!",
    "body": "Mr. Wayne, I found your secret identity! I know you're Batman! There’s no denying it."
}

spam_email = {
    "sender": "Crypto bro",
    "subject": "The best investment of 2025",
    "body": "Mr Wayne, I just launched an ALT coin and want you to buy some!"
}

print("\nProcessing legitimate email...")
compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

print("\nProcessing spam email...")
compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})


Processing legitimate email...
Alfred is proccessing email from         Joker with subject:             Found you Batman!
Classification result: a most...unsettling email, indeed. after careful consideration, i must conclude that this email is:

**spam**

the tone and content of the email are far too brazen and lacking in subtlety to be considered legitimate. the joker's penchant for dramatic flair and unpredictability makes it unlikely that he would reveal such a crucial piece of information so casually. furthermore, the email's brevity and lack of context raise suspicions about its authenticity.

as mr. wayne's trusted butler, i must remain vigilant in protecting his secrets and detecting potential threats. this email, i fear, is merely a clever ruse designed to unsettle our beloved dark knight.
Alfred has marked the email as spam.
The email has been movd to the spam folder.

Processing spam email...
Alfred is proccessing email from         Crypto bro with subject:             The b

{'email': {'sender': 'Crypto bro',
  'subject': 'The best investment of 2025',
  'body': 'Mr Wayne, I just launched an ALT coin and want you to buy some!'},
 'is_spam': True,
 'spam_reason': None,
 'email_category': None,
 'messages': []}